In [8]:
import json
import re
import sys
import gzip
import codecs
import string
from math import log2
from collections import Counter
from spacy.lang.en import English
from traceback_with_variables import activate_by_import
import nltk
import string

In [9]:
def filter_punctuation(ngrams):
    # Input: assume ngrams is a list of ['token1','token2'] bigrams
    # Removes ngrams like ['today','.'] where either token is a single punctuation character
    # Note that this does not mean tokens that merely *contain* punctuation, e.g. "'s"
    # Returns list with the items that were not removed
    punct = string.punctuation
    return [ngram.lower()   for ngram in ngrams   if ngram not in punct]


In [23]:
def ngrams(tokens, n):
    grams=[]
    for i in range(1,len(tokens)):
       gram = tokens[i-1:i+n-1]
       if len(gram)==n:
            grams.append(gram)
    return grams

In [71]:
R='However, I consider that racist, sexist or other discriminatory language, and allegations against Members, are offensive.'
A='However, I consider racist language, sexist or other discrimination, and allegations against Members to be offensive.'
B='However, I regard racist language, language that discriminates on the basis of sex or on any other grounds, and allegations against Members, as offensive.'
C='Racist Members consider that discriminatory allegations as language are the basis of offensive sexist allegations, however.'
D='Allegations against members are offensive.'

In [61]:
def clean_tokens(line,n):
    nlp= English(parser=False) # faster init with parse=False, if only using for tokenization
    doc1 = nlp(line)
    word_tokens=[]
    for token in doc1:
        word_tokens.append(token.text)
    normalized_tokens = filter_punctuation(word_tokens)
    sents = ngrams(normalized_tokens, n)
    new_sents = [sent for sent_sublist in sents for sent in sent_sublist]
    return new_sents

### (R,A)

In [65]:
one_n_R = clean_tokens(R,1)
print(one_n_R)
one_n_C = clean_tokens(A,1)
print(one_n_C)

['however', 'i', 'consider', 'that', 'racist', 'sexist', 'or', 'other', 'discriminatory', 'language', 'and', 'allegations', 'against', 'members', 'are']
['however', 'i', 'consider', 'racist', 'language', 'sexist', 'or', 'other', 'discrimination', 'and', 'allegations', 'against', 'members', 'to', 'be']


In [70]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
reference = [one_n_R] # list of references for 1 sentence. # list of references for all sentences in corpus.
candidate = one_n_C # list of hypotheses that corresponds to list of references.
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

Individual 1-gram: 0.800000
Individual 2-gram: 0.500000
Individual 3-gram: 0.307692
Individual 4-gram: 0.083333


### (R,B)

In [72]:
one_n_R = clean_tokens(R,1)
print(one_n_R)
one_n_C = clean_tokens(B,1)
print(one_n_C)

['however', 'i', 'consider', 'that', 'racist', 'sexist', 'or', 'other', 'discriminatory', 'language', 'and', 'allegations', 'against', 'members', 'are']
['however', 'i', 'regard', 'racist', 'language', 'language', 'that', 'discriminates', 'on', 'the', 'basis', 'of', 'sex', 'or', 'on', 'any', 'other', 'grounds', 'and', 'allegations', 'against', 'members', 'as']


In [73]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
reference = [one_n_R] # list of references for 1 sentence. # list of references for all sentences in corpus.
candidate = one_n_C # list of hypotheses that corresponds to list of references.
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

Individual 1-gram: 0.478261
Individual 2-gram: 0.181818
Individual 3-gram: 0.095238
Individual 4-gram: 0.050000


### (R,C)

In [74]:
one_n_R = clean_tokens(R,1)
print(one_n_R)
one_n_C = clean_tokens(C,1)
print(one_n_C)

['however', 'i', 'consider', 'that', 'racist', 'sexist', 'or', 'other', 'discriminatory', 'language', 'and', 'allegations', 'against', 'members', 'are']
['racist', 'members', 'consider', 'that', 'discriminatory', 'allegations', 'as', 'language', 'are', 'the', 'basis', 'of', 'offensive', 'sexist', 'allegations']


In [75]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
reference = [one_n_R] # list of references for 1 sentence. # list of references for all sentences in corpus.
candidate = one_n_C # list of hypotheses that corresponds to list of references.
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

Individual 1-gram: 0.600000
Individual 2-gram: 0.071429
Individual 3-gram: 0.000000
Individual 4-gram: 0.000000


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)


### (R,D)

In [76]:
one_n_R = clean_tokens(R,1)
print(one_n_R)
one_n_C = clean_tokens(D,1)
print(one_n_C)

['however', 'i', 'consider', 'that', 'racist', 'sexist', 'or', 'other', 'discriminatory', 'language', 'and', 'allegations', 'against', 'members', 'are']
['allegations', 'against', 'members', 'are']


In [77]:
import nltk
from nltk.translate.bleu_score import sentence_bleu
reference = [one_n_R] # list of references for 1 sentence. # list of references for all sentences in corpus.
candidate = one_n_C # list of hypotheses that corresponds to list of references.
print('Individual 1-gram: %f' % sentence_bleu(reference, candidate, weights=(1, 0, 0, 0)))
print('Individual 2-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 1, 0, 0)))
print('Individual 3-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 1, 0)))
print('Individual 4-gram: %f' % sentence_bleu(reference, candidate, weights=(0, 0, 0, 1)))

Individual 1-gram: 0.063928
Individual 2-gram: 0.063928
Individual 3-gram: 0.063928
Individual 4-gram: 0.063928
